In [1]:
import sys
sys.path.append('../../src/')

import pandas as pd
import numpy as np
import networkx as nx
import scipy.sparse as sp
from stellargraph import StellarDiGraph

from extract_network import extract_network

2021-12-15 11:51:40.386210: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-15 11:51:44.661621: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12966 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:18:00.0, compute capability: 7.5
2021-12-15 11:51:44.663252: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13524 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:19:00.0, compute capability: 7.5
2021-12-15 11:51:44.664764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 1352

## HeadToHead

In [2]:
data = pd.read_csv('./HeadToHead_Slim.csv', header=None)

In [3]:
data

,0,1,2,3
0,281884,50,2726956381513270,4
1,281884,50,2741700308864760,8
2,281900,20,2731647631125110,3
3,281900,20,2740359951832990,6
4,281902,50,2737009059253030,0
...,...,...,...,...
12449,290747,20,2734998523705230,5
12450,290803,50,2732987988157080,5
12451,290803,50,2729637095577350,6
12452,290956,50,2729637095577350,3


In [4]:
games = list(set(data[0].values))
games

[281884,
 281900,
 281902,
 281903,
 281906,
 281911,
 281914,
 281920,
 281921,
 281925,
 281926,
 281927,
 281930,
 281934,
 281935,
 281941,
 281943,
 281944,
 281945,
 281946,
 281947,
 281950,
 281951,
 281952,
 281957,
 281959,
 281962,
 281964,
 281966,
 281969,
 281970,
 281972,
 281973,
 281975,
 281976,
 281977,
 281979,
 281980,
 281982,
 281984,
 281986,
 281987,
 281988,
 281990,
 281996,
 281999,
 282002,
 282003,
 282004,
 282005,
 282008,
 282009,
 282011,
 282015,
 282017,
 282020,
 282022,
 282023,
 282024,
 282029,
 282031,
 282033,
 282036,
 282037,
 282038,
 282039,
 282040,
 282042,
 282044,
 282045,
 282048,
 282050,
 282052,
 282054,
 282055,
 282056,
 282059,
 282061,
 282062,
 282063,
 282065,
 282068,
 282069,
 282070,
 282071,
 282072,
 282073,
 282074,
 282075,
 282078,
 282079,
 282080,
 282082,
 282084,
 282086,
 282087,
 282090,
 282092,
 282093,
 282094,
 282095,
 282096,
 282098,
 282100,
 282101,
 282102,
 282106,
 282107,
 282109,
 282110,
 282111,
 

In [5]:
data[data[0]==281900][3].values[0]

3

In [6]:
df = pd.DataFrame(columns=['source', 'target', 'weight'])

In [7]:
i = 0
for game in games:
    sub_df = data[data[0]==game]
    scores = sub_df[3].values
    players = sub_df[2].values
    if scores[0] > scores[1]:
        df.loc[i] = [players[0], players[1], scores[0] - scores[1]]
        i += 1
    elif scores[0] < scores[1]:
        df.loc[i] = [players[1], players[0], scores[1] - scores[0]]
        i += 1

In [8]:
df

,source,target,weight
0,2741700308864760,2726956381513270,4
1,2740359951832990,2731647631125110,3
2,2736338880736940,2737009059253030,2
3,2739019594800950,2737679237769100,3
4,2728296738545260,2734328345189110,1
...,...,...,...
5711,2733658166673120,2732987988157220,5
5712,2727626560029320,2732987988157220,1
5713,2732987988157080,2734998523705230,2
5714,2729637095577350,2732987988157080,1


In [9]:
df['weight'] = df['weight'].astype(np.float32)

In [10]:
dataset = 'HeadToHead'
graph = StellarDiGraph(edges=df)
print(graph.info())
adj = graph.to_adjacency_matrix(weighted=True)
assert np.abs(adj-adj.transpose()).sum() != 0
node_list = list(graph.nodes())
print(adj.shape)
adj, node_list = extract_network(adj, np.array(node_list), lowest_degree=1)
print(adj.shape)
sp.save_npz(dataset+'adj', adj)
np.save(dataset+'node_list', node_list)
print(dataset+' done!')

StellarDiGraph: Directed multigraph
 Nodes: 602, Edges: 5716

 Node types:
  default: [602]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [5716]
        Weights: range=[1, 39], mean=4.19052, std=2.68802
        Features: none
(602, 602)
(602, 602)
HeadToHead done!
